In [1]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [2]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [3]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [4]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-01 17:30:33 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-01 17:30:34 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-01 17:30:34 INFO Request successful
2023-05-01 17:30:34 DEBUG 


<Response [200]>

In [5]:
ask_ki = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    },
    graph_pattern='''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .''',
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
)
ask_ki

2023-05-01 17:30:34 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-01 17:30:34 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-01 17:30:34 INFO Request successful
2023-05-01 17:30:34 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/c61cd94d-dc03-4153-bdd4-ecfaed7e7221"}


{'knowledgeInteractionId': 'http://127.0.0.1:8000/subscriber/interaction/c61cd94d-dc03-4153-bdd4-ecfaed7e7221',
 'prefixes': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'saref': 'https://w3id.org/saref#'}}

In [6]:
ask_ki_id = ask_ki["knowledgeInteractionId"]
ask_ki_id

'http://127.0.0.1:8000/subscriber/interaction/c61cd94d-dc03-4153-bdd4-ecfaed7e7221'

In [7]:
ask_ki_prefixes = ask_ki["prefixes"]
ask_ki_prefixes

{'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'saref': 'https://w3id.org/saref#'}

In [8]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]

2023-05-01 17:30:35 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-01 17:30:35 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 829
2023-05-01 17:30:35 INFO Request successful
2023-05-01 17:30:35 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"23.5","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/1>","timestamp":"\"2023-05-01T17:30:31+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/52669df0-af2b-43a9-b843-96894b99c0fb","exchangeStart":"2023-05-01T15:30:35.382+00:00","exchangeEnd":"2023-05-01T15:30:35.580+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"23.5","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measur

In [67]:
binding_set

[{'room_id': '"http://127.0.0.1:8000/thermostat/rooms/1"',
  'temp': '23.5',
  'device_id': '"http://127.0.0.1:8000/thermostat/devices/1"',
  'meas': '<http://127.0.0.1:8000/thermostat/measurements/1>',
  'timestamp': '"2023-05-01T17:30:31+02:00"'}]

In [68]:
def replace_values_in_graph_pattern(graph_pattern, binding_set):
    for binding in binding_set:
        for key, value in binding.items():
            graph_pattern = graph_pattern.replace('?' + key, value)
    # Remove < and > characters from graph_pattern
    graph_pattern = graph_pattern.replace('<', '').replace('>', '')
    graph_pattern = graph_pattern.splitlines()
    graph_pattern = [line.strip() for line in graph_pattern if line.strip()]
    return graph_pattern

In [69]:
data=replace_values_in_graph_pattern(
    graph_pattern='''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .''',
    binding_set=binding_set)
data


['http://127.0.0.1:8000/thermostat/measurements/1 rdf:type saref:Measurement .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:hasValue 23.5 .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:isMeasuredIn saref:TemperatureUnit .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:hasTimestamp "2023-05-01T17:30:31+02:00" .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:isMeasurementOf "http://127.0.0.1:8000/thermostat/rooms/1" .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:relatesToProperty saref:Temperature .',
 'http://127.0.0.1:8000/thermostat/measurements/1 saref:measurementMadeBy "http://127.0.0.1:8000/thermostat/devices/1" .']

In [70]:
def replace_prefixes_with_uris(graph_pattern, prefixes):
    result = []
    for line in graph_pattern:
        for prefix, uri in prefixes.items():
            line = line.replace(prefix + ':',uri).replace('"','')
        result.append(line)
    return result

In [71]:
data=replace_prefixes_with_uris(
    graph_pattern=data,
    prefixes=ask_ki_prefixes)
data

['http://127.0.0.1:8000/thermostat/measurements/1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/saref#Measurement .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#hasValue 23.5 .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#isMeasuredIn https://w3id.org/saref#TemperatureUnit .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#hasTimestamp 2023-05-01T17:30:31+02:00 .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#isMeasurementOf http://127.0.0.1:8000/thermostat/rooms/1 .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#relatesToProperty https://w3id.org/saref#Temperature .',
 'http://127.0.0.1:8000/thermostat/measurements/1 https://w3id.org/saref#measurementMadeBy http://127.0.0.1:8000/thermostat/devices/1 .']

In [73]:
from rdflib import Graph, URIRef
from rdflib.plugins.stores import sparqlstore

def save_graph_to_graphdb(graph, readURL, writeURL):
    mystore = sparqlstore.SPARQLUpdateStore()
    mystore.open((readURL, writeURL))
    mystore.add_graph(graph)

def convert_to_turtle_rdf(data):
    result = []
    for line in data:
        subject, predicate, obj = line.split()[:3]
        if obj.startswith('http'):
            obj = f'<{obj}>'
        else:
            obj = f'"{obj}"'
        result.append(f'<{subject}> <{predicate}> {obj} .')
    return '\n'.join(result)

def store_data_in_graphdb(data, readURL, writeURL):
    turtle_rdf = convert_to_turtle_rdf(data)
    print(turtle_rdf)
    g = Graph(identifier=URIRef("http://example.org/mygraph"))
    g.parse(data=turtle_rdf, format="turtle")
    g.print()
    save_graph_to_graphdb(g, readURL, writeURL)

readURL = "https://graphdb.odissei.nl/repositories/MateuszTest"
writeURL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

store_data_in_graphdb(data, readURL, writeURL)

<http://127.0.0.1:8000/thermostat/measurements/1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/saref#Measurement> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#hasValue> "23.5" .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#isMeasuredIn> <https://w3id.org/saref#TemperatureUnit> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#hasTimestamp> "2023-05-01T17:30:31+02:00" .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#isMeasurementOf> <http://127.0.0.1:8000/thermostat/rooms/1> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#relatesToProperty> <https://w3id.org/saref#Temperature> .
<http://127.0.0.1:8000/thermostat/measurements/1> <https://w3id.org/saref#measurementMadeBy> <http://127.0.0.1:8000/thermostat/devices/1> .
@prefix ns1: <https://w3id.org/saref#> .

<http://127.0.0.1:8000/thermostat/measurements/1> a ns1:Measurement ;